In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip -q install langchain tiktoken pypdf sentence_transformers InstructorEmbedding faiss-cpu
!pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 4.4 MB/s eta 0:00:00


In [ ]:
!pip install -U pip
!pip install -U pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 124.9 MB/s eta 0:00:00


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain.embeddings import HuggingFaceEmbeddings
import os
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader
from langchain.schema import Document
from dotenv import load_dotenv
from collections import defaultdict
from typing import List
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field, field_validator
from typing import List, Optional
from langchain_core.output_parsers import PydanticOutputParser
import re


In [ ]:
def get_files(directory_path='/content/drive/MyDrive/Resume CV_Uploaded/'):
    loader = DirectoryLoader(
        path=directory_path,
        loader_cls=PyMuPDFLoader,   # ✅ No parentheses
        show_progress=True,
        glob="**/*.pdf",
        use_multithreading=True
    )
    docs = loader.load()
    return docs


In [ ]:
docs = get_files()
print(docs[0].page_content)


100%|██████████| 3/3 [00:00<00:00, 13.75it/s]

MUHAMMAD ANIS FASEEL
MUHAMMAD ANIS FASEEL
MUHAMMAD ANIS FASEEL
Sultan town  (Near by Ali town Station), Lahore
malikanas4737@gmail.com | +923166461053
AI engineer with hands-on experience in Generative AI, Machine Learning, and Data Analysis. Skilled at building simple
AI models, working with Large Language Models (LLMs), and understanding data to improve business tasks.
Comfortable using tools like Python, TensorFlow, scikit-learn, and creating visual charts for data analysis. Always eager
to learn new tools and use AI to solve real-world problems.
University of South Asia, Lahore 
Bachelor of Software Engineering
Related Coursework: Database Systems, Computer Science, Operations Management, Management, Business
Software Development, AI And Machine Learning
GPA: 3.00/4.00
EDUCATION
   Laptop Price Prediction Using Machine Learning
Developed a Machine Learning model to predict laptop prices based on features such as brand, specifications, and
other relevant attributes. Utilized regress

In [ ]:
def merge_pages(docs):
  combined_texts=defaultdict(str)
  for doc in docs:
    source=doc.metadata.get("source","unknown")
    combined_texts[source] += doc.page_content.strip() + "\n"
    merged_docs = [
          Document(page_content=text, metadata={"source": source})
          for source, text in combined_texts.items()
      ]
  print(f"Length of the Merge data {len(merged_docs)}")
  return merged_docs



In [ ]:
merged_docs = merge_pages(docs)

Length of the Merge data 3


In [ ]:
len(merged_docs)

3

In [ ]:
def get_embedding(merged_docs):
  embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
  vector_store=FAISS.from_documents(merged_docs,embedding)
  return vector_store

In [ ]:
  # Convert distance to similarity score
  def distance_to_score(distance: float) -> float:
      """Convert L2 distance to a normalized similarity score (0–100%)"""
      similarity = 1 / (1 + distance)         # Normalized between 0 and 1
      return round(similarity * 100, 2)

In [ ]:
    def ret_chain(vector_store):
        retriever_chain = RunnableLambda(
            lambda query: [
                _add_score_to_doc(doc, score)
                for doc, score in vector_store.similarity_search_with_score(query)
            ]
        )
        return retriever_chain

In [ ]:
      def _add_score_to_doc(doc: Document, score: float) -> Document:
          score_percent = distance_to_score(score)
          doc.metadata["score_percent"] = score_percent
          doc.metadata["distance"] = score
          return doc

In [ ]:
def label_score_by_distance(distance: float) -> str:
    if distance <= 0.8:
        return "Strong match"
    elif distance <= 1.2:
        return "Moderate match"
    else:
        return "Weak match"

In [ ]:
class Education(BaseModel):
    university_name: str = Field(..., description="Name of the university or institution")
    degree: str = Field(..., description="Degree earned by the candidate")
    gpa: Optional[str] = Field(None, description="GPA or grade, if available")


class Experience(BaseModel):
    company_name: str = Field(..., description="Name of the company or organization")
    n_years: Optional[str] = Field(None, description="Duration in years at this company")
    project_name: Optional[str] = Field(None, description="Name or title of a key project")
    project_description: Optional[str] = Field(None, description="Brief description of the project")
    tech_stack: List[str] = Field(..., description="Technologies and tools used in the project")

    @field_validator("tech_stack", mode="before")
    def handle_not_found_tech_stack(cls, v):
        if isinstance(v, str) and v.strip().lower() == "not found":
            return []
        return v


class ContactInfo(BaseModel):
    email: Optional[str] = Field(None, description="Email address")
    phone: Optional[str] = Field(None, description="Phone number")
    linkedin: Optional[str] = Field(None, description="LinkedIn profile URL")


class ResumeReport(BaseModel):
    name: str = Field(..., description="Full name of the candidate")
    # age: Optional[int] = Field(None, ge=0, description="Age of the candidate, if available")
    # native_languages: List[str] = Field(default_factory=list, description="List of native languages spoken by the candidate")
    # match_score: int = Field(..., ge=0, le=100, description="Match score between 0–100")
    # match_score: str = Field(..., description="Match score between 0–100")
    education: Optional[List[Education]] = Field(..., description="List of education details")
    experience: Optional[List[Experience]] = Field(..., description="List of professional experience")
    skills: Optional[List[str]] = Field(..., description="List of key skills and strengths")
    employment_gaps: Optional[str] = Field(None, description="Employment gaps if found")
    contact_info: ContactInfo = Field(..., description="Contact information")
    summary: str = Field(..., description="Short summary about the candidate's fit for the job")

    @field_validator("skills", mode="before")
    def convert_skills_not_found(cls, v):
        if isinstance(v, str) and v.strip().lower() == "not found":
            return []
        return v

    @field_validator("education", "experience", mode="before")
    def convert_list_fields_not_found(cls, v):
        if isinstance(v, str) and v.strip().lower() == "not found":
            return []
        return v

In [ ]:
def get_metadata(doc):
  distance = round(float(doc.metadata['distance']),4)
  score = round(float(doc.metadata['score_percent']),4)
  source = doc.metadata['source']
  return {
      'distance': distance,
      'score': score,
      'source': source
  }

def extract_years_from_text(text: str) -> float:
    """
    Extracts a numeric value (float/int) from a string and converts months to years if needed.

    Args:
        text (str): e.g., '6-12 months', '0.5 years', '1 year'

    Returns:
        float: Duration in years
    """
    match = re.search(r"\d+(\.\d+)?", text)
    if not match:
        return 0.0

    value = float(match.group())

    if 'month' in text.lower():
        return round(value / 12, 2)
    else:
        return round(value, 2)


In [ ]:
def get_all_cv_results(retriever_docs, main_chain, job_description):
  RESULTS = []
  for retriever in retriever_docs:
    res = main_chain.invoke({"job_description": job_description, "resume": retriever.page_content}).dict()
    metadata = get_metadata(retriever)
    res['metadata'] = metadata
    res['label'] = label_score_by_distance(metadata['distance'])
    res['match_score'] = metadata['score']
    total = 0
    for exp in res['experience']:
      yrs = extract_years_from_text(exp['n_years'])
      total += yrs
    res['total_experience'] = total
    RESULTS.append(res)
  return RESULTS

In [ ]:
  def get_results(directory_path, job_description):
      docs = get_files(directory_path)
      merged_docs = merge_pages(docs)
      vector_store = get_embedding(merged_docs)
      retriever_chain = ret_chain(vector_store)

      llm = ChatGroq(temperature = 0, model="llama3-8b-8192",
                    groq_api_key="GROQ_API_KEY")

      retriever = retriever_chain.invoke(job_description)

      resume_parser = PydanticOutputParser(pydantic_object=ResumeReport)

      resume_prompt = PromptTemplate(
      template="""
          You are a smart and precise Resume Analyzer.

          You are given a **Job Description** and a candidate's **Resume** (plain text).
          Your task is to analyze how well the resume matches the job requirements and return a structured report as a VALID JSON object.

          ONLY use the resume content for your answers.
          If any information is missing from the resume, write "Not Found".

          ---

          Job Description:
          {job_description}

          Resume:
          {resume}

          ---

          Return ONLY a JSON object in the format below:
          {format_instructions}
          """,
              input_variables=["job_description", "resume"],
              partial_variables={"format_instructions": resume_parser.get_format_instructions()}
          )

      main_chain =  resume_prompt | llm | resume_parser

      RESULTS = get_all_cv_results(retriever, main_chain, job_description)

      return RESULTS

In [ ]:
  directory_path = "/content/drive/MyDrive/Resume CV_Uploaded"
  job_description = '''
            Job Title: Data Scientist
            We are seeking a Data Scientist with a strong foundation in data analysis, machine learning, and statistical modeling. The ideal candidate will be responsible for extracting insights from complex datasets, building predictive models, and communicating findings that drive business decisions.

            Key Responsibilities
            Analyze large datasets to discover trends, patterns, and actionable insights.

            Build and deploy machine learning models for classification, regression, and clustering tasks.

    '''
  res = get_results(directory_path, job_description)
  print(res)

100%|██████████| 3/3 [00:00<00:00, 45.66it/s]


Length of the Merge data 3


/tmp/ipython-input-41-1386176070.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  res = main_chain.invoke({"job_description": job_description, "resume": retriever.page_content}).dict()
/tmp/ipython-input-41-1386176070.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  res = main_chain.invoke({"job_description": job_description, "resume": retriever.page_content}).dict()


[{'name': 'Muhammad Talha', 'education': [{'university_name': 'National Textile University', 'degree': 'Bachelor of Software Engineering', 'gpa': '3.52'}], 'experience': [{'company_name': 'Freelance', 'n_years': 'Present', 'project_name': 'Document Classifier', 'project_description': 'Automated classification of title closing documents using BERT model', 'tech_stack': ['TensorFlow', 'AWS Lambda', 'API Gateway']}, {'company_name': 'TransData', 'n_years': '1 year 3 months', 'project_name': 'Document Classifier', 'project_description': "Developed and implemented the 'Document Classifier' at TransData, an advanced AI system for automating the classification of title closing documents in the mortgage and lending industry", 'tech_stack': ['TensorFlow', 'AWS Lambda', 'API Gateway']}, {'company_name': 'BLING', 'n_years': '6 months', 'project_name': 'Message Spam Detection and Sentiment Analysis', 'project_description': 'Contributed to projects related to message spam detection and sentiment an

/tmp/ipython-input-41-1386176070.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  res = main_chain.invoke({"job_description": job_description, "resume": retriever.page_content}).dict()


In [ ]:
print(res)

[{'name': 'Muhammad Talha', 'education': [{'university_name': 'National Textile University', 'degree': 'Bachelor of Software Engineering', 'gpa': '3.52'}], 'experience': [{'company_name': 'Freelance', 'n_years': 'Present', 'project_name': 'Document Classifier', 'project_description': 'Automated classification of title closing documents using BERT model', 'tech_stack': ['TensorFlow', 'AWS Lambda', 'API Gateway']}, {'company_name': 'TransData', 'n_years': '1 year 3 months', 'project_name': 'Document Classifier', 'project_description': "Developed and implemented the 'Document Classifier' at TransData, an advanced AI system for automating the classification of title closing documents in the mortgage and lending industry", 'tech_stack': ['TensorFlow', 'AWS Lambda', 'API Gateway']}, {'company_name': 'BLING', 'n_years': '6 months', 'project_name': 'Message Spam Detection and Sentiment Analysis', 'project_description': 'Contributed to projects related to message spam detection and sentiment an

In [ ]:
import pandas as pd

In [ ]:
df=pd.DataFrame(res)

In [ ]:
df

,name,education,experience,skills,employment_gaps,contact_info,summary,metadata,label,match_score,total_experience
0,Muhammad Talha,[{'university_name': 'National Textile Univers...,"[{'company_name': 'Freelance', 'n_years': 'Pre...","[Machine Learning, Data Analysis, Deep Learnin...",None,"{'email': 'muhammadtalha1818@gmail.com', 'phon...",Highly experienced in the domains of machine l...,"{'distance': 0.8122, 'score': 55.18, 'source':...",Moderate match,55.18,1.16
1,MUHAMMAD ANIS FASEEL,[{'university_name': 'University of South Asia...,"[{'company_name': 'Self-Initiated Projects', '...","[Generative AI, Machine Learning, Deep Learnin...",None,"{'email': 'malikanas4737@gmail.com', 'phone': ...",AI engineer with hands-on experience in Genera...,"{'distance': 1.0823, 'score': 48.02, 'source':...",Moderate match,48.02,0.00
2,John Doe,[{'university_name': 'University of California...,"[{'company_name': 'XYZ Technologies', 'n_years...","[Manual Testing, Automation Testing, Test Plan...",Not Found,"{'email': 'Not Found', 'phone': 'Not Found', '...",Detail-oriented Software Testing Engineer with...,"{'distance': 1.4443, 'score': 40.91, 'source':...",Weak match,40.91,5.00
